In [1]:
#https://github.com/crawfordsm/specidentify/blob/master/docs/notebook/arcwidget_tutorial.ipynb

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
# imports needed from 
import numpy as np
from matplotlib import pyplot as plt

from astropy import units as u
from astropy import modeling as mod
from astropy.io import fits
from astropy.table import Table

from specutils import Spectrum1D

In [3]:
# imports needed from specidentify 
from specidentify import WavelengthSolution
from specidentify import ArcIdentify

In [4]:
lamp_image = fits.getdata('mbxgpP202109140035.fits')
lamp_spec = (np.median(lamp_image[448:458, :], axis=0))

arc = Spectrum1D(spectral_axis=np.arange(len(lamp_spec))*u.pixel , flux=lamp_spec*u.electron)

In [5]:
# a plot of the uncalibrated arc spectra
ax = plt.subplots()[1]  
ax.plot(arc.spectral_axis, arc.flux)  
ax.set_xlabel("Dispersion [pixels]")  
ax.set_ylabel("Flux [electron]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Flux [electron]')

In [6]:
xarr = np.arange(len(arc.data))

line_table = np.loadtxt('Argon_lores.dat.txt', unpack=True, usecols=(0,1))
line_table = Table(line_table.T, names=('wavelength', 'flux'))

In [7]:
ws_init = mod.models.Chebyshev1D(3)
ws_init.domain = [xarr.min(), xarr.max()]
ws_init = mod.fitting.LinearLSQFitter()(ws_init, xarr, xarr)

In [8]:
ws = WavelengthSolution(x=None, wavelength=None, model=ws_init)

In [9]:
x = [841,923,1352,1656,1767,1836,2219,2359]
w = [3948,4200,5495,6416,6752,6965,8115,8521]
ws = WavelengthSolution(x=x, wavelength=w, model=ws_init)
ws.fit()

In [10]:
aw = ArcIdentify(arc, ws, line_table, wavelength_range=[3800, 9000], flatten_order=9)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
aw.show_commands()


 x - centroid on line    a - Display spectrum
 b - identify features   f - fit solution
 r - redraw spectrum     e - add closest line
 d - delete feature
       


In [12]:
aw.draw_error()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Displays the rms of the fit
aw.ws.sigma(aw.xp, aw.wp)

1.9496604812834448

In [14]:
arc_w = Spectrum1D(spectral_axis=aw.ws(np.arange(len(lamp_spec))) * u.angstrom, flux=lamp_spec*u.electron)

In [15]:
ax = plt.subplots()[1]  
ax.plot(arc_w.spectral_axis, arc_w.flux)  
ax.set_xlabel("Dispersion [Angstrom]")  
ax.set_ylabel("Flux [electron]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Flux [electron]')

In [65]:
f = open('ref_ar_pg300_gr5.dat','w')
[f.write('%g %g\n' % (arc_w.spectral_axis[i].angstrom, lamp_spec[i])) for i in range(len(arc_w.spectral_axis))]
f.close()